In [1]:
import pandas as pd
import numpy as np
import json
import os
import Capstone_functions as cf
import capstone_json_functions as cjf
import sql_functions as sf
from pathlib import Path
import re


Global variables

In [2]:
directory = "data/kickstarter_json/"

schema = 'bgg_data'
engine = sf.get_engine()


## First try for one file
Rename files, if they start with "Kickstarter..."

In [ ]:
cjf.rename_kickstarter_files(directory)

Read the file and store it into a list called "lines"

In [3]:
# lines is a list of strings
filename = "2022-06-09.json"
file_path = f"{directory}{filename}"
lines = cjf.list_of_lines(file_path)

build a dictionairy by looking at the "category"

In [ ]:
#list_of_categories = cjf.list_of_categories(file_path)

Build the dataframe

In [4]:
df = cjf.build_df_from_lines_list(lines)

In [ ]:
df.columns

List of Columns to keep

In [9]:
columns_to_keep = cjf.columns_to_keep

In [15]:
columns_to_keep[0] = 'name'

In [16]:
columns_to_keep

['name',
 'goal',
 'pledged',
 'state',
 'slug',
 'country',
 'currency',
 'deadline',
 'created_at',
 'launched_at',
 'backers_count',
 'usd_pledged',
 'converted_pledged_amount',
 'fx_rate',
 'usd_exchange_rate',
 'current_currency',
 'usd_type',
 'creator_name',
 'state_changed_at']

Overwrite the df with itself with the chosen columns

In [17]:
df = df[columns_to_keep]

Convert Unix timestamp to pandas datetime

In [18]:
df = cjf.unix_to_datetime(df)

/Users/ruperthufnagel/DailyLama/Daily-Lama-Capstone/capstone_json_functions.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_datetime(df[col],unit='s')
/Users/ruperthufnagel/DailyLama/Daily-Lama-Capstone/capstone_json_functions.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_datetime(df[col],unit='s')
/Users/ruperthufnagel/DailyLama/Daily-Lama-Capstone/capstone_json_functions.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4129 entries, 0 to 0
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   name                      4129 non-null   object        
 1   goal                      4129 non-null   int64         
 2   pledged                   4129 non-null   float64       
 3   state                     4129 non-null   object        
 4   slug                      4129 non-null   object        
 5   country                   4129 non-null   object        
 6   currency                  4129 non-null   object        
 7   deadline                  4129 non-null   datetime64[ns]
 8   created_at                4129 non-null   datetime64[ns]
 9   launched_at               4129 non-null   datetime64[ns]
 10  backers_count             4129 non-null   int64         
 11  usd_pledged               4129 non-null   object        
 12  converted_pledged_amoun

In [34]:
df.usd_pledged = df.usd_pledged.astype(float)

In [33]:
mask = df["fx_rate"] != df["usd_exchange_rate"]
df.loc[mask,'usd_pledged'] / df.loc[mask,"pledged"]
#df["checkor"] = df.usd_pledged / df.pledged

,name,goal,pledged,state,slug,country,currency,deadline,created_at,launched_at,backers_count,usd_pledged,converted_pledged_amount,fx_rate,usd_exchange_rate,current_currency,usd_type,creator_name,state_changed_at,checkor
0,Folk Horrors by Ana Polanšćak,1000,20306.00,successful,folk-horrors-by-ana-polanscak,GB,GBP,2022-06-07 07:56:54,2022-05-16 15:20:51,2022-05-31 07:56:54,435,25678.963539,25397,1.256735,1.250716,USD,international,Andrew May,2022-06-07 07:56:54,1.264600
0,Hunters of the Lost Creatures - The Card Game,3998,34054.00,successful,hunters-of-the-lost-creatures-the-card-game-0,DE,EUR,2022-06-07 17:00:00,2021-04-08 17:44:47,2022-05-10 16:59:12,1587,35790.633108,36375,1.068374,1.068170,USD,international,Wonderbow Games,2022-06-07 17:00:00,1.050996
0,PANZER COLLECTION - Tabletop Games STL files f...,2900,9916.80,successful,panzer-collection-tabletop-games-stl-files-for...,MX,MXN,2022-05-25 03:24:07,2022-05-01 21:20:58,2022-05-10 03:24:07,108,492.393218,497,0.050940,0.050210,USD,international,Klaus,2022-05-25 03:24:07,0.049652
0,MK VI Landship Modular 3D Printable Tank Kit,300,3637.29,successful,treadheads-modular-3d-printable-tank-kit,CA,CAD,2022-05-31 19:58:00,2022-04-22 20:03:37,2022-05-09 18:35:51,205,2817.748002,2869,0.796859,0.789011,USD,international,Tread Heads,2022-05-31 19:58:00,0.774683
0,"Pocket Landship, No Man's Land",1000,88653.50,successful,pocket-landship-no-mans-land,GB,GBP,2022-05-31 22:00:00,2021-05-01 11:16:41,2022-05-03 22:49:22,2146,111335.419960,111845,1.256735,1.261602,USD,international,Word Forge Games,2022-05-31 22:00:00,1.255849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Dodoresque Cherry Blossom - Miniature Addon 2021,200,1169.00,successful,dodoresque-cherry-blossom-miniature-addon-2021,DE,EUR,2021-12-10 02:00:00,2021-11-12 12:02:54,2021-12-02 15:29:49,51,1324.713442,1325,1.068374,1.133667,USD,international,Tetrahedron Games,2021-12-10 02:00:00,1.133202
0,Demons of the Hollowdepths - Dragonbond Daimos,100000,530542.60,successful,demons-of-the-hollowdepths,MX,MXN,2021-12-16 20:00:00,2021-10-22 01:37:55,2021-11-30 14:00:22,478,24315.902719,25178,0.050940,0.047459,USD,international,Draco Studios,2021-12-16 20:00:00,0.045832
0,Combat Lifter - Take your tabletop to the next...,5000,9729.29,successful,combat-lifter-take-your-tabletop-to-the-next-l...,CA,CAD,2022-01-28 20:42:16,2021-10-14 03:09:15,2021-11-29 20:42:16,102,7606.356003,7640,0.796859,0.785340,USD,international,Yeti Miniatures,2022-01-28 20:42:16,0.781800
0,Fantasy Advent Calendar 2021,1000,5017.00,successful,fantasy-advent-calendar-2021,GB,GBP,2021-12-27 18:00:44,2021-11-06 10:10:24,2021-11-25 18:00:44,292,6710.161693,6726,1.256735,1.340838,USD,international,The Oracle,2021-12-27 18:00:44,1.337485


In [36]:
df.sort_values("usd_exchange_rate")

,name,goal,pledged,state,slug,country,currency,deadline,created_at,launched_at,backers_count,usd_pledged,converted_pledged_amount,fx_rate,usd_exchange_rate,current_currency,usd_type,creator_name,state_changed_at,checkor
0,『オレとオマエの異世界転生』リブート版を作りたい！,600000,5655709.0,live,1374645271,JP,JPY,2022-06-25 14:59:00,2022-04-28 08:26:25,2022-05-25 10:01:30,483,44260.45,42223,0.007466,0.007466,USD,international,蜂月@六角えんぴつ,2022-05-25 10:01:31,0.007826
0,World of Pratheron : Dragon,200000,277809.0,live,world-of-pratheron-dragon,JP,JPY,2022-06-24 07:19:07,2022-05-24 13:43:11,2022-05-25 07:19:07,60,2174.08,2073,0.007466,0.007466,USD,international,YOUNGHO SON,2022-05-25 07:19:08,0.007826
0,時計仕掛けの館/Clockwork House,330000,202129.0,live,clockwork-house,JP,JPY,2022-06-24 13:00:00,2022-02-14 02:00:53,2022-05-25 10:38:39,43,1581.82,1509,0.007466,0.007466,USD,international,トリノザッカ/torinozaca,2022-05-25 10:38:40,0.007826
0,Deathly Thrones,12000,155348.0,live,deathly-thrones,JP,JPY,2022-06-30 13:00:00,2022-05-25 03:31:06,2022-05-31 17:00:16,248,1223.40,1159,0.007466,0.007466,USD,international,Josselin Loizeau,2022-05-31 17:00:17,0.007875
0,ドール工房と気まぐれな魔女,300000,617009.0,live,1023058165,JP,JPY,2022-06-30 14:59:00,2022-02-20 12:15:08,2022-05-21 15:00:03,68,4834.79,4606,0.007466,0.007466,USD,international,ジョイアブル,2022-05-21 15:00:04,0.007836
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Ghost Club RPG,250,4483.0,successful,ghostclubrpg,GB,GBP,2022-01-17 11:52:07,2021-06-12 03:38:56,2021-12-18 11:52:07,545,5974.23,6132,1.256735,1.367858,USD,international,Andrew Dowell,2022-01-17 11:52:07,1.332640
0,The Excelsior Collection - Quality Affordable ...,250,304.0,successful,the-excelsior-collection-quality-affordable-ha...,GB,GBP,2022-01-17 07:00:00,2021-12-07 04:50:30,2022-01-03 17:10:09,8,411.36,415,1.254115,1.367858,USD,international,Landmark Dice,2022-01-17 07:00:00,1.353151
0,The Excelsior Collection - Quality Affordable ...,250,304.0,successful,the-excelsior-collection-quality-affordable-ha...,GB,GBP,2022-01-17 07:00:00,2021-12-07 04:50:30,2022-01-03 17:10:09,8,411.36,415,1.256735,1.367858,USD,international,Landmark Dice,2022-01-17 07:00:00,1.353151
0,Dragon Map Maker: TTRPG battle maps made easy,20000,49962.0,successful,dragon-map-maker-ttrpg-battle-maps-made-easy,GB,GBP,2022-01-13 21:16:43,2021-06-01 10:26:27,2021-12-14 21:16:43,1485,66195.70,68516,1.256735,1.371381,USD,international,Darren Rolmanis (Dragon Maps Ltd),2022-01-13 21:16:43,1.324921


# And now with all the json files in the folder data/kickstarter_json

In [ ]:
pathlist = Path(directory).glob('**/*.json')
df_all = pd.DataFrame()
counter = 0
for path in pathlist:
    file_path = str(path)
    print(file_path)
    lines = cjf.list_of_lines(file_path)
    df_temp = cjf.build_df_from_lines_list(lines)
    df_all = pd.concat([df_all, df_temp])
    counter += 1
print(f"{counter} files have been dataframed")


In [ ]:
df = cjf.build_df_from_all_files_in_dir()

In [ ]:
df.info()

In [ ]:
sf.build_table(engine=engine,table_name='Kickstarter_raw',dataframe=df,schema='bgg_data')

In [ ]:
columns_to_keep = cjf.columns_to_keep
df_all = df_all[columns_to_keep]
df_all = cjf.unix_to_datetime(df_all)
df_all.rename({"name":"game_name"},axis=1,inplace=True)
df_all = df_all.astype({'usd_pledged': f"{'float64'}"}).round(2)
df_all['game_name'] =  df_all['game_name'].str.findall(r'\w|\s').str.join('').str.replace(r"\s+","_").str.lower()


In [ ]:
df_clean = cjf.clean_df(df)

In [ ]:
df_clean.info()

In [ ]:
df = df_clean.drop_duplicates()

In [ ]:
df.info()

In [ ]:
df2 = df.copy()

In [ ]:
schema = 'bgg_data'
main_table_name = 'main'
name_table = 'game_names_masked'

In [ ]:
main_query = f"SELECT * FROM {schema}.{main_table_name};"
name_query = f"SELECT * FROM {schema}.{name_table};"

In [ ]:
df_main_bgg = sf.get_dataframe(main_query)
df_name_bgg = sf.get_dataframe(name_query)

In [ ]:
df_name_bgg.nunique()

In [ ]:
df_name_bgg.info()

In [ ]:
df_name_bgg.drop_duplicates(inplace=True)

In [ ]:
df_name_bgg.drop("primary_bool",axis=1,inplace=True)

In [ ]:
df_name_bgg.drop_duplicates(inplace=True)

In [ ]:
df_name_bgg.duplicated('game_name').sum()

In [ ]:
df_temp = df_name_bgg.drop_duplicates("game_name")

In [ ]:
df_temp.duplicated("game_name").sum()

In [ ]:
df_ks_with_bgg_id = pd.merge(df_temp,df,how='inner',on='game_name',validate='1:m')

In [ ]:
df_ks_with_bgg_id.nunique()

In [ ]:
df_ks_with_bgg_id["overall_successful"] = df_ks_with_bgg_id["state"] == 'successful'

In [ ]:
df_ks_with_bgg_id.sort_values("slug").head(30)

Which id was successful in the end?

In [ ]:
df_ks_with_bgg_id.groupby("id")["successful"].sum()

In [ ]:
df_successful = df_ks_with_bgg_id.where(df_ks_with_bgg_id["successful"])